# Tutorial with examples

In [2]:
from cmfa.fluxomics_data.reaction import Reaction
from cmfa.fluxomics_data.reaction_network import ReactionNetwork
from cmfa.data_preparation import import_fluxomics_dataset_from_json
from cmfa.fluxomics_data.compound import AtomPattern
from cmfa.fluxomics_data.emu import EMU


In [3]:
import os

CUR_DIR = os.getcwd()
TEST_DIR = f'{CUR_DIR}/../data/test_data'

TEST_DIR

'/Users/te/Library/CloudStorage/OneDrive-Personal/Biosustain/cmfa/cmfa/../data/test_data'

In [4]:
model = import_fluxomics_dataset_from_json(f'{TEST_DIR}/model.json')

In [5]:
model.reaction_network.reactions

{Reaction id=R1, name=R1,stoichiometry={'A': {abc: -1.0}, 'B': {abc: 1.0}},reversible=False, ,
 Reaction id=R2, name=R2,stoichiometry={'B': {abc: -1.0}, 'D': {abc: 1.0}},reversible=True, ,
 Reaction id=R3, name=R3,stoichiometry={'B': {abc: -1.0}, 'C': {bc: 1.0}, 'E': {a: 1.0}},reversible=False, ,
 Reaction id=R4, name=R4,stoichiometry={'B': {abc: -1.0}, 'C': {de: -1.0}, 'D': {bcd: 1.0}, 'E': {a: 1.0, e: 1.0}},reversible=False, ,
 Reaction id=R5, name=R5,stoichiometry={'D': {abc: -1.0}, 'F': {abc: 1.0}},reversible=False, }

In [6]:
print(model.reaction_network.reaction_adjacency_matrix())

compound           A       B     C           D               E           F
pattern          abc     abc    bc  de     abc   bcd         a     e   abc
compound pattern                                                          
A        abc      []    [R1]    []  []      []    []        []    []    []
B        abc      []      []  [R3]  []  [R2_f]  [R4]  [R4, R3]    []    []
C        bc       []      []    []  []      []    []        []    []    []
         de       []      []    []  []      []  [R4]        []  [R4]    []
D        abc      []  [R2_r]    []  []      []    []        []    []  [R5]
         bcd      []      []    []  []      []    []        []    []    []
E        a        []      []    []  []      []    []        []    []    []
         e        []      []    []  []      []    []        []    []    []
F        abc      []      []    []  []      []    []        []    []    []


In [7]:
df = model.reaction_network.reaction_adjacency_matrix()
column_data = df[('F', 'abc')]

# Step 2: Filter for non-empty cells
# Assuming non-empty cells contain lists with at least one element, and empty cells contain empty lists.
non_empty_cells = column_data[column_data.apply(lambda x: len(x)>0)]

# Step 3: Retrieve the non-empty values and their corresponding rows
for index, value in non_empty_cells.items():
    print(f"Row: {index}, Value: {value}")

Row: ('D', 'abc'), Value: ['R5']


In [22]:
import pandas as pd
import re
from collections import deque

def _extract_atom_pattern(compound_pattern: str):
    """Extract the atom pattern from a compound pattern string.

    Parameters
    ----------
    compound_pattern : str
        The compound pattern string, e.g., "Glu(abcde)".

    Returns
    -------
    str
        The extracted atom pattern, e.g., "abcde".
    """
    match = re.search(r'(.+)\((.+)\)', compound_pattern)
    if match:
        compound, pattern = match.groups()
        return compound, pattern 
    else:
        raise ValueError(f"No atom pattern found in the input: {compound}")

def decompose_network(
    target_compound: str, reaction_network: ReactionNetwork
) -> pd.DataFrame:
    

    MAM = reaction_network.reaction_adjacency_matrix()
    all_cpd = MAM.columns.get_level_values(0).unique()

    queue = deque([target_compound])
    visited = set()
    emu_maps = {}

    while queue:
        # Keep poping new EMU added from previous round.
        cur_emu = queue.popleft()
        print(f"Current EMU: {cur_emu}, have visited {visited}")
        if cur_emu in visited:
            continue
        visited.add(cur_emu)

        # Pattern matching
        _cur_emu = EMU(id="_".join(cur_emu), compound=cur_emu[0], pattern=AtomPattern(pattern=cur_emu[1]))

        emu_size = _cur_emu.size
        if emu_size not in emu_maps:
            emu_maps[emu_size] = {}
        
        # Find the reaction that is participated in the current emu
        col = MAM[cur_emu]
        reactants = col[col.apply(lambda x: len(x)>0)]

        # Finding multiple reactants
        _reactants_str = reactants.apply(lambda x: str(x))
        multi_reactants = _reactants_str.duplicated(keep=False)

        if multi_reactants.any():
            # Use the duplicates mask to filter the original Series and get the multi-index keys
            duplicate_keys = reactants[multi_reactants].index.tolist()

            overlap_pat = find_matchable_parts(duplicate_keys, cur_emu[1])

            for new_emu in overlap_pat:
                queue.append(new_emu)
        else:
            # Add reactants to the queue and emu map
            print(reactants, "\n", )
            for i in reactants.index:
                print(i)
                match_cpd_pat = [key for key in MAM.index if key[0] == i[0]]
                for m in match_cpd_pat:
                    queue.append(tuple(m))
                # Add a condition to stop when the number of atoms are not matching -> Stop when this happens(B23 + C1)
            
        for next_emu in reactants:
            emu_maps[emu_size].setdefault(cur_emu, []).append(next_emu)

            
    return emu_maps


def find_matchable_parts(keys, match):
    new_keys = []
    for key in keys:
        # Unpack the tuple
        letter, sequence = key
        
        # Check for matchable parts and keep characters that are found in the matchable part
        matched_sequence = ''.join([char for char in sequence if char in match])
        
        # Construct a new tuple with the matched parts
        new_key = (letter, matched_sequence)
        new_keys.append(new_key)
        
    return new_keys

In [23]:
decompose_network(('F','abc'), model.reaction_network)

Current EMU: ('F', 'abc'), have visited set()
compound  pattern
D         abc        [R5]
Name: (F, abc), dtype: object 

('D', 'abc')
Current EMU: ('D', 'abc'), have visited {('F', 'abc')}
compound  pattern
B         abc        [R2_f]
Name: (D, abc), dtype: object 

('B', 'abc')
Current EMU: ('D', 'bcd'), have visited {('D', 'abc'), ('F', 'abc')}
Current EMU: ('B', 'abc'), have visited {('D', 'abc'), ('D', 'bcd'), ('F', 'abc')}
compound  pattern
A         abc          [R1]
D         abc        [R2_r]
Name: (B, abc), dtype: object 

('A', 'abc')
('D', 'abc')
Current EMU: ('B', 'bc'), have visited {('D', 'abc'), ('D', 'bcd'), ('F', 'abc'), ('B', 'abc')}


KeyError: ('B', 'bc')

In [ ]:
MAM = model.reaction_network.reaction_adjacency_matrix()
all_cpd = MAM.columns.get_level_values(0).unique()
selected_cpd_pat = [key for key in MAM.index if key[0] == 'C' ]
selected_cpd_pat

[('C', 'bc'), ('C', 'de')]

In [ ]:
# Alternative method from freeflux
import numpy as np
def decompose_network(ini_emus, lump = True, n_jobs = 1):
    '''
    Parameters
    ----------
    ini_emus: dict
        Metabolite ID => atom NOs or list of atom NOs. Atom NOs can be int list or str, 
        e.g., {'Ala': [[1,2,3], '23'], 'Ser': '123'}
    lump: bool
        Whether to lump linear EMUs.
    n_jobs: int
        # of jobs to run in parallel.
        
    Returns
    -------
    mergedEAMs: dict of df
        Size => merged EMU adjacency matrix (EAM)

    Notes
    -----
    EMUs in sequential reactions can not be lumped in transient MFA.      
    '''
    
    metabids = []
    atom_nos = []
    for metabid, atomNOs in ini_emus.items():
        if isinstance(atomNOs, list):
            if any(isinstance(item, Iterable) for item in atomNOs):
                for atomnos in atomNOs:
                    if not isinstance(atomnos, str):
                        atomnos = ''.join(map(str, atomnos))
                    metabids.append(metabid)
                    atom_nos.append(atomnos)
            else:
                atomNOs = ''.join(map(str, atomNOs))
                metabids.append(metabid)
                atom_nos.append(atomnos)
        else:
            metabids.append(metabid)
            atom_nos.append(atomNOs)            
    
    return _decompose_network(metabids, atom_nos, lump = lump, n_jobs = n_jobs) 


def _decompose_network(metabolites, atom_nos, lump = True):
    '''
    Parameters
    ----------
    metabolites: list of str
        List of metabolite IDs from which initial EMU will be generated to start the 
        decomposition.
    atom_nos: list of str
        Atom NOs of corresponding metabolites, len(atom_nos) should be equal to 
        len(metabolites).
    lump: bool
        Whether to lump linear EMUs.    
    n_jobs: int
        # of jobs to run in parallel.
    
    Returns
    -------
    mergedEAMs: dict of df
        Size => merged EMU adjacency matrix (EAM).

    Notes
    -----
    EMUs in sequential reactions can not be lumped in transient MFA.    
    '''
    
    emus = []
    for metabid, atomNOs in zip(metabolites, atom_nos):
        emus.append(EMU(metabid+'_'+atomNOs, Metabolite(metabid), atomNOs))
        
    EAMsAll = []
    for emu in emus:
        EAMs = get_emu_adjacency_matrices(emu, lump)
        EAMsAll.append(EAMs)
    
    mergedEAMs = _merge_all_EAMs(*EAMsAll)
    
    return mergedEAMs

def _merge_all_EAMs(self, *EAMsAll):
    '''
    Parameters
    ----------
    EAMsAll: tuple of EAMs
        EAMs is dict of DataFrame, i.e., Size => EMU adjacency matrix (EAM).

    Returns
    -------    
    mergedEAMs: dict of df
        Size => merged EAM     
    '''
    
    mergedEAMs = {}
    maxsize = max([max(EAMs) for EAMs in EAMsAll])
    for size in range(1, maxsize+1):
        EAMCurrentSize = list(
            filter(
                lambda x: isinstance(x, pd.DataFrame), 
                [EAMs.get(size, 0) for EAMs in EAMsAll]
            )
        )
        if EAMCurrentSize:   
            mergedEAMs[size] = reduce(self._merge_EAMs, EAMCurrentSize)
    
    return mergedEAMs            

    

def get_emu_adjacency_matrices(self, iniEMU, lump = True):
        '''
        Parameters
        ----------
        iniEMU: EMU
            Starting EMU of the decomposition.
        lump: bool
            Whether to lump linear EMUs.
        
        Returns
        -------
        EAMs: dict of df
            Size => EMU adjacency matrix (EAM) after lumping of linear EMUs and combination
            of equivalent EMUs. Index and columns are EMUs, cells are symbolic expression of flux.

        Notes
        -----
        EMUs in sequential reactions can not be lumped in transient MFA.
        '''
            
        oriEAMs = _get_original_EAMs(iniEMU)
        
        if lump:
            lumpedEAMs = _lump_linear_EMUs(oriEAMs, iniEMU)
            combinedEAMs = _combine_equivalent_EMUs(lumpedEAMs)
        else:
            combinedEAMs = _combine_equivalent_EMUs(oriEAMs)
        
        return combinedEAMs

def _get_original_EAMs(self, iniEMU):
        '''
        Parameters
        ----------
        iniEMU: EMU
            Starting EMU of the decomposition.
            
        Returns
        -------
        EAMs: dict of df
            Size => original EMU adjacency matrix (EAM), cells are symbolic expression of flux。
        '''
        
        EAMsInfo = _BFS(iniEMU)
        
        EAMs = {}
        for size, EMUsInfo in EAMsInfo.items():
            
            nonSourceEMUs = set([EMUInfo[0] for EMUInfo in EMUsInfo])
            SourceEMUs = sorted(
                set(
                    [tuple(EMUInfo[1]) if len(EMUInfo[1]) > 1 else EMUInfo[1][0] 
                     for EMUInfo in EMUsInfo]
                ) - nonSourceEMUs
            )
            
            EAM = pd.DataFrame(
                Integer(0), 
                index = sorted(nonSourceEMUs) + sorted(SourceEMUs), 
                columns = sorted(nonSourceEMUs)
            )
            for emu, preEMUs, flux in EMUsInfo:
                
                col = emu
                
                if len(preEMUs) == 1:
                    idx = preEMUs[0]
                else:
                    idx = tuple(preEMUs)
                
                EAM.loc[[idx], col] += flux   
            
            EAMs[size] = EAM
        
        return EAMs

def _BFS(self, iniEMU):
        '''
        Parameters
        ----------
        iniEMU: EMU
            Starting EMU of the decomposition. 
            Metabolite of iniEMU can be any Metabolite instance with the same id.
        
        Returns
        -------
        EAMsInfo: dict of list
            Size => list of [EMU, [precursor EMUs], symbolic expression of flux].
        '''
        
        MAM = self.metabolite_adjacency_matrix
        
        EAMsInfo = {}
        
        searched = []
        toSearch = deque()
        toSearch.appendleft(iniEMU)
        while toSearch:
            
            currentEMU = toSearch.pop()
            searched.append(currentEMU)
                    
            formingRxns = list(
                set(chain(*[cell for cell in MAM[currentEMU.metabolite_id] if cell]))
            )   
            for formingRxn in formingRxns:
                
                if formingRxn.reversible:
                    if currentEMU.metabolite_id in formingRxn.products_with_atoms:
                        asProMetabs = formingRxn.products_info['metab'][currentEMU.metabolite_id]
                        direction = 'forward'
                        flux = formingRxn.fflux
                    else:
                        asProMetabs = formingRxn.substrates_info['metab'][currentEMU.metabolite_id]
                        direction = 'backward'
                        flux = formingRxn.bflux
                else:
                    asProMetabs = formingRxn.products_info['metab'][currentEMU.metabolite_id]
                    direction = 'forward'
                    flux = formingRxn.flux
                
                if isinstance(asProMetabs, pd.Series):
                    offset = 1/asProMetabs.size
                    asProMetabs = list(asProMetabs)
                else:
                    offset = 1.0
                    asProMetabs = [asProMetabs]
                
                
                for asProMetab in asProMetabs:    
                    currentEMU = EMU(currentEMU.id, asProMetab, currentEMU.atom_nos)   
                    preEMUsInfo = formingRxn._find_precursor_EMUs(currentEMU, direction = direction)
                    
                    for preEMUs, coe in preEMUsInfo:
                        for preEMU in preEMUs:
                            if preEMU not in searched and preEMU not in toSearch:
                                toSearch.appendleft(preEMU)
                            
                        EAMsInfo.setdefault(currentEMU.size, []).append(
                            [currentEMU, preEMUs, offset * coe * flux]
                        )
                            
        return EAMsInfo

def _find_precursor_EMUs(self, emu, direction = 'forward'):
    '''
    Parameters
    ----------
    emu: EMU
    direction: str
        * For reversible reaction,
        'forward' if emu is product and precursor emu(s) are substrates;
        'backward' if emu is substrate and precursor emu(s) are products.
        * For irreversible reaction, only 'forward' is acceptable.
        
    Returns
    -------
    preEMUsInfo: list

    Notes
    -----
    For reaction like: A({'ab': 0.5, 'ba': 0.5}) + B({'c': 1}) -> C({'abc': 0.5, 'cba': 0.5}),
    _find_precursor_EMUs(C12) returns
    [[[A_12], 0.5],
        [[B_1, A_2], 0.25],
        [[B_1, A_1], 0.25]].
    '''        
    
    if self.reversible:
        if direction == 'forward':
            atomMapping = self._substrates_atom_mapping
            reacInfo = self.substrates_info['metab']
        
        elif direction == 'backward':
            atomMapping = self._products_atom_mapping
            reacInfo = self.products_info['metab']
    
    else:
        if direction != 'forward':
            raise ValueError('only "forward" is acceptable for irreversible reaction')
        
        atomMapping = self._substrates_atom_mapping
        reacInfo = self.substrates_info['metab']
    
    
    preEMUsInfoRaw = []
    for scenario in atomMapping:
        for atoms, coe in emu.metabolite.atoms_info.items():   
            
            preAtoms = {}
            uniCoe = coe
            for atom in [atoms[no-1] for no in emu.atom_nos]:
                
                pre, preAtomNO, preCoe = scenario[atom]   
                
                if pre not in preAtoms:
                    uniCoe *= preCoe
                    preAtoms[pre] = [preAtomNO]
                else:
                    preAtoms[pre].append(preAtomNO)
            
            preEMUs = [EMU(pre.id+'_'+''.join(map(str, sorted(preAtomNOs))), pre, preAtomNOs) 
                        for pre, preAtomNOs in preAtoms.items()]
            preEMUsInfoRaw.append([preEMUs, uniCoe])
    
    preEMUsInfoRaw = [Counter({tuple(sorted(preEMUs)): coe}) 
                        for preEMUs, coe in preEMUsInfoRaw]   
    preEMUsInfo = reduce(lambda x, y: x+y, preEMUsInfoRaw) 
    preEMUsInfo = [[list(preEMUs), coe] for preEMUs, coe in preEMUsInfo.items()]
    
    return preEMUsInfo
    

def _lump_linear_EMUs(self, EAMs, iniEMU):
    '''
    Parameters
    ----------
    EAMs: dict of df
        Size => original EMU adjacency matrix (EAM), cells are symbolic expression of flux.
    iniEMU: EMU
        Starting EMU of the decomposition.
    
    Returns
    -------
    lumpedEAMs: dict of df
        Size => lumped EMU adjacency matrix (EAM), cells are symbolic expression of flux.
    '''
    
    lumpedEAMs = {}
    orderedSizes = sorted(EAMs.keys(), reverse = True)
    for i, size in enumerate(orderedSizes):
        
        lumpedEAM = EAMs[size].copy(deep = 'all')
        for emu in lumpedEAM.columns:
            
            preEMUs = lumpedEAM.index[lumpedEAM[emu] != 0]
            if preEMUs.size == 1:   
                preEMU = preEMUs[0]
                
                if (emu != iniEMU and 
                    (preEMU not in lumpedEAM.columns or lumpedEAM.loc[emu, preEMU] == 0)):
                    
                    lumpedEAM.drop(emu, axis = 1, inplace = True)
                    
                    lumpedEAM.index = self._replace_list_item(lumpedEAM.index, emu, preEMU)
                    
                    for j in range(i):   
                        largerEAM = lumpedEAMs[orderedSizes[j]]
                        largerEAM.index = self._replace_list_item(largerEAM.index, emu, preEMU)
                    
                    lumpedEAM = _uniquify_dataFrame_index(lumpedEAM)   
                    upper = _uniquify_dataFrame_index(
                        lumpedEAM.loc[lumpedEAM.columns, :]
                    )
                    lower = _uniquify_dataFrame_index(
                        lumpedEAM.loc[lumpedEAM.index.difference(lumpedEAM.columns),:]
                    )
                    lumpedEAM = pd.concat((upper, lower))
        
        lumpedEAMs[size] = lumpedEAM
        
    return lumpedEAMs
    
    
def _combine_equivalent_EMUs(self, EAMs):
    '''
    Parameters
    ----------
    EAMs: dict of df
        Size => original EMU adjacency matrix (EAM), cells are symbolic expression of flux.
        
    Returns
    -------
    combinedEAMs: dict df
        Size => EAM with equivalent EMUs combined, cells are symbolic expression of flux.
    '''
    
    combinedEAMs = {}
    for size, EAM in EAMs.items():
        
        combinedEAM = EAM.copy(deep = 'all')
        combined = []
        for emu in combinedEAM.columns:
            if emu not in combined:
                
                equivEMU = emu.equivalent
                if equivEMU in combinedEAM.columns:   
                    
                    combinedEAM.loc[:, emu] = combinedEAM.loc[:, [emu, equivEMU]].sum(axis = 1)/2
                    combinedEAM.drop(equivEMU, axis = 1, inplace = True)
                    
                    combinedEAM.loc[emu, :] = combinedEAM.loc[[emu, equivEMU], :].sum()
                    combinedEAM.drop(equivEMU, inplace = True)
                    
                    combined.append(equivEMU)
                
        combinedEAMs[size] = combinedEAM

    return combinedEAMs

def _uniquify_dataFrame_index(self, df):
    '''
    Parameters
    ----------
    df: df
        DataFrame to be uniquify.
        
    Returns
    -------
    uniqueDf: df
        DataFrame with duplicate rows combined (summated).
    '''
    
    sortedDf = df.sort_index()

    sortDfIndex, idx = np.unique(sortedDf.index, return_index = True)

    uniqueDf = pd.DataFrame(
        np.add.reduceat(sortedDf.values, idx), 
        index = sortDfIndex, 
        columns = sortedDf.columns
    )
    
    return uniqueDf

In [ ]:
'''Define the EMU class.'''


__author__ = 'Chao Wu'
__date__ = '02/26/2022'


from functools import lru_cache
from collections.abc import Iterable
from .metabolite import Metabolite


class EMU():
    '''
    Define EMU (i.e., elementary metabolite unit) object and its operations.

    EMUs in the same metabolite and with the same atom NOs are considered as identical, 
    while metabolites which they derived from could be different.
    
    EMUs can be compared based self.metabolite_id and self.atom_nos.
    EMU and iterable object of EMUs can also be compared. In this case EMU will be put into
    the same iterable object with single item, and comparison between two iterables are performed.
    
    Currently only binary equivalents are considered.
    
    Parameters
    ----------
    id: str
        EMU ID
    metabolite: Metabolite or str
        Which metabolite the EMU comes from.
    atom_nos: list of int or str
        Atom NOs, sorted by number.

    Attributes
    ----------
    id: str
        EMU ID
    metabolite: Metabolite
        Which metabolite the EMU comes from.    
    metabolite_id: str
        Metabolite ID.
    atom_nos: list of int
        Atom NOs, sorted by number.
    size: int
        Size of EMU.
    equivalent_atom_nos: None or list of int
        Equivalent atom NOs, sorted by number.   
    equivalent: EMU
        Equivalent of EMU.
    '''
    
    def __init__(self, id, metabolite, atom_nos):
        '''
        Parameters
        ----------
        id: str
            EMU ID
        metabolite: Metabolite or str
            Which metabolite the EMU comes from.
        atom_nos: list of int or str
            Atom NOs, sorted by number.
        '''
        
        self.id = id
        if isinstance(metabolite, Metabolite):
            self.metabolite = metabolite
            self.metabolite_id = self.metabolite.id
        elif isinstance(metabolite, str):
            self.metabolite = Metabolite(metabolite)
            self.metabolite_id = metabolite
        if isinstance(atom_nos, list):
            self.atom_nos = sorted(atom_nos)
        elif isinstance(atom_nos, str):
            self.atom_nos = sorted(list(map(int, atom_nos)))
        self.size = len(self.atom_nos)
        
    
    def __hash__(self):
        
        return hash(self.metabolite_id) + hash(sum(self.atom_nos))
        
        
    def __eq__(self, other):
        '''
        Parameters
        ----------
        other: EMU or iterable
        '''
        
        if isinstance(other, Iterable):
            return type(other)([self]) == other
        else:
            return self.metabolite_id == other.metabolite_id and self.atom_nos == other.atom_nos
        
        
    def __lt__(self, other):
        '''
        Parameters
        ----------
        other: EMU or iterable
        '''
        
        if isinstance(other, Iterable):
            return type(other)([self]) < other
        else:
            if self.metabolite_id != other.metabolite_id:
                return self.metabolite_id < other.metabolite_id
            else:
                return self.atom_nos < other.atom_nos
                
    
    def __gt__(self, other):
        '''
        Parameters
        ----------
        other: EMU or iterable
        '''
        
        if isinstance(other, Iterable):
            return type(other)([self]) > other
        else:
            if self.metabolite_id != other.metabolite_id:
                return self.metabolite_id > other.metabolite_id
            else:
                return self.atom_nos > other.atom_nos
        
    
    @property
    @lru_cache()
    def equivalent_atom_nos(self):
        '''
        Returns
        -------
        equivAtomNOs: list of int or None
            Equivalent atom NOs, sorted by number.
        '''
        
        if len(self.metabolite.atoms_info) == 1:
            return None

        else:
            refAtoms, equivAtoms = self.metabolite.atoms_info   
            
            mapping = dict(zip(refAtoms, range(1, len(refAtoms)+1)))
            
            equivAtomNOs = sorted([mapping[equivAtoms[no-1]] for no in self.atom_nos])
            
            if equivAtomNOs == self.atom_nos:
                return None
            else:
                return equivAtomNOs
                
                
    @property
    @lru_cache()
    def equivalent(self):
        '''
        Returns
        -------
        EMU: EMU
            Equivalent of current EMU.
        '''
        
        equivAtomNOs = self.equivalent_atom_nos
        
        if equivAtomNOs:
            id = self.metabolite_id + '_' + ''.join(map(str, self.equivalent_atom_nos))
            metab = self.metabolite
            return EMU(id, metab, equivAtomNOs)
        else:
            return None
            
        
    def __repr__(self):
        
        return f'{self.__class__.__name__} {self.metabolite_id}_{"".join(map(str, self.atom_nos))}'
    
        